# 09. Isomap

측지선 거리를 이용한 비선형 차원축소를 구현합니다.

## 학습 목표
- 측지선(geodesic) 거리 이해
- 그래프 기반 거리 계산
- MDS와 Isomap의 관계

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(42)

## 1. 스위스롤 데이터

In [ ]:
# 스위스롤 생성
n_samples = 1000

t = 1.5 * np.pi * (1 + 2 * torch.rand(n_samples))
height = 10 * torch.rand(n_samples)

X = torch.stack([
    t * torch.cos(t),
    height,
    t * torch.sin(t)
], dim=1)

colors = t.numpy()

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=colors, cmap='viridis', s=10)
ax.set_title('Swiss Roll')
plt.show()

## 2. 유클리드 거리 vs 측지선 거리

- **유클리드 거리**: 직선 거리 (3D 공간에서)
- **측지선 거리**: 매니폴드 위를 따라 가는 거리

In [ ]:
# 두 점 선택하여 거리 비교
idx1, idx2 = 0, 500

euclidean_dist = torch.norm(X[idx1] - X[idx2]).item()
print(f"Point 1 (t={t[idx1]:.2f}): {X[idx1].numpy()}")
print(f"Point 2 (t={t[idx2]:.2f}): {X[idx2].numpy()}")
print(f"\nEuclidean distance: {euclidean_dist:.2f}")
print(f"True geodesic (via t): {abs(t[idx1] - t[idx2]):.2f}")

## 3. Isomap 적용

Isomap 알고리즘:
1. 이웃 그래프 구축
2. 최단 경로로 측지선 거리 근사 (Floyd-Warshall/Dijkstra)
3. MDS로 저차원 임베딩

In [ ]:
from mlfs.classical.reduction import Isomap, PCA

# Isomap 적용
isomap = Isomap(n_components=2, n_neighbors=10)
X_isomap = isomap.fit_transform(X)

# PCA 비교
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=colors, cmap='viridis', s=10)
axes[0].set_title('PCA')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')

axes[1].scatter(X_isomap[:, 0], X_isomap[:, 1], c=colors, cmap='viridis', s=10)
axes[1].set_title('Isomap')
axes[1].set_xlabel('Isomap 1')
axes[1].set_ylabel('Isomap 2')

plt.tight_layout()
plt.show()

## 4. 이웃 수의 영향

In [ ]:
n_neighbors_list = [5, 10, 20, 50]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for ax, n_neighbors in zip(axes, n_neighbors_list):
    iso = Isomap(n_components=2, n_neighbors=n_neighbors)
    X_reduced = iso.fit_transform(X)
    
    ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, cmap='viridis', s=5)
    ax.set_title(f'n_neighbors = {n_neighbors}')

plt.suptitle('Isomap: Effect of Neighbors', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 구멍이 있는 데이터

Isomap은 데이터에 구멍이 있어도 잘 작동합니다.

In [ ]:
# 구멍이 있는 스위스롤
n = 1500
t_full = 1.5 * np.pi * (1 + 2 * torch.rand(n))
h_full = 10 * torch.rand(n)

# 구멍 만들기 (특정 영역 제거)
mask = ~((t_full > 6) & (t_full < 8) & (h_full > 3) & (h_full < 7))
t_hole = t_full[mask]
h_hole = h_full[mask]

X_hole = torch.stack([
    t_hole * torch.cos(t_hole),
    h_hole,
    t_hole * torch.sin(t_hole)
], dim=1)

colors_hole = t_hole.numpy()

# 시각화
fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(X_hole[:, 0], X_hole[:, 1], X_hole[:, 2], c=colors_hole, cmap='viridis', s=5)
ax1.set_title('Swiss Roll with Hole (3D)')

ax2 = fig.add_subplot(132)
pca_h = PCA(n_components=2)
X_pca_h = pca_h.fit_transform(X_hole)
ax2.scatter(X_pca_h[:, 0], X_pca_h[:, 1], c=colors_hole, cmap='viridis', s=5)
ax2.set_title('PCA')

ax3 = fig.add_subplot(133)
iso_h = Isomap(n_components=2, n_neighbors=12)
X_iso_h = iso_h.fit_transform(X_hole)
ax3.scatter(X_iso_h[:, 0], X_iso_h[:, 1], c=colors_hole, cmap='viridis', s=5)
ax3.set_title('Isomap (Preserves Hole)')

plt.tight_layout()
plt.show()

## 6. LLE vs Isomap 비교

In [ ]:
from mlfs.classical.reduction import LLE

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# PCA
axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=colors, cmap='viridis', s=5)
axes[0].set_title('PCA (Linear)')

# LLE
lle = LLE(n_components=2, n_neighbors=12)
X_lle = lle.fit_transform(X)
axes[1].scatter(X_lle[:, 0], X_lle[:, 1], c=colors, cmap='viridis', s=5)
axes[1].set_title('LLE (Local)')

# Isomap
axes[2].scatter(X_isomap[:, 0], X_isomap[:, 1], c=colors, cmap='viridis', s=5)
axes[2].set_title('Isomap (Global)')

plt.suptitle('Comparison of Dimensionality Reduction Methods', fontsize=14)
plt.tight_layout()
plt.show()

## 요약

1. **Isomap**: 측지선 거리를 보존하는 비선형 차원축소
2. **핵심**: 이웃 그래프 + 최단 경로 + MDS
3. **장점**: 전역 구조 보존, 구멍이 있어도 작동
4. **LLE와 비교**: LLE는 지역, Isomap은 전역 구조 중시

다음: **앙상블 학습** (AdaBoost)